# Read SEQ-file plot, ...

## imports

In [ ]:
import logging
#logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)
import numpy as np
from numpy import r_
import scipy as sp
import scipy.signal
import scipy.misc
import datetime
import os
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%config InlineBackend.figure_formats = ['svg']
#%matplotlib inline
#%matplotlib notebook 
%matplotlib qt
#%matplotlib widget

mpl.rcParams['figure.figsize'] = r_[250, 120]/25.4
mpl.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.constrained_layout.use'] = True

## Installation of the fnv module:
* Download and install the "FLIR Science File SDK" from https://flir.custhelp.com/app/account/fl_download_software.
* Install Python module according https://flir.custhelp.com/app/answers/detail/a_id/3504/~/getting-started-with-flir-science-file-sdk-for-python  
    Attention: Use the setup.py attached to the "Getting started with ..."  as the one in the SDK is broken.   

In [ ]:
## load SEQfileReader
import sys
# download form https://github.com/marthoch/SEQfileReader/
sys.path.insert(0, r'C:\data\GitHub\SEQfileReader\src')
sys.path.insert(0, r'..\PythonLib\SEQfileReader\src')
from SEQfileReader import SEQfileReader

## load data

In [ ]:
filename = r'test.seq'

In [ ]:
seq = SEQfileReader(filename)
seq

## read lines

In [ ]:
#lineparam = dict(p0=dict(v=27, h=16), p1=dict(v=46, h=634))
lineparam = dict(p0=dict(v=27, h=16+8), p1=dict(v=46, h=634-8))

In [ ]:
## show line placement
seq.go2frame(-1)
fig = seq.read_line(line=lineparam, plot_show_line=True)
fig.set_size_inches(r_[250, 60]/25.4)
fig.suptitle('{s.filename_woext}: fnr={s.current_frame_number}, time={s.current_frame_time} '.format(s=seq))
fig

In [ ]:
## read full data
#%timeit -r1 -n1 
lines_over_time_df = seq.read_line_over_time_df(line=lineparam)

In [ ]:
## plot data
param = dict(cmap='plasma') #, vmin=365-273.15, vmax=375-273.15)
fig,ax = plt.subplots(1,1) #,figsize=r_[300, 70]/25.4)
p = ax.imshow(lines_over_time_df.pixels.to_numpy()-273.15, interpolation='nearest', origin='lower', **param)
fig.set_size_inches(r_[300, 150]/25.4)
fig.suptitle('{s.filename_woext}: time={s.first_frame_time} '.format(s=seq))
None
fig

In [ ]:
## plot data presentation
param = dict(cmap='plasma') #, vmin=365-273.15, vmax=375-273.15)
fig,ax = plt.subplots(1,1) #,figsize=r_[300, 70]/25.4)
p = ax.imshow(lines_over_time_df.pixels.to_numpy()-273.15, interpolation='nearest', origin='lower', **param)
ax.set_axis_off()
fig.set_size_inches(r_[300, 150]/25.4)
#fig.suptitle('{s.filename_woext}: time={s.first_frame_time} '.format(s=seq))
#ax.set_xlim([2200,4800])
#fig.savefig('presenation_web.png', dpi=600)
None
fig

In [ ]:
lines_over_time_df.set_index([('time', 'timestamp')], inplace=True)
#lines_over_time_df.set_index([('time', 'time_sec')], inplace=True)

In [ ]:
fig, ax = plt.subplots(1,1)
lines_over_time_df.pixels[12].plot(ax=ax)
fig